# 🎨 Stable Diffusion XL Image Generator
Generate high-quality images with Stable Diffusion XL (SDXL) 1.0 using Colab’s free GPU.

In [ ]:
# ─── Setup ────────────────────────────────────────────────────────────
# 1) Clear old pip cache
!pip cache purge

# 2) Install compatible versions in one shot:
!pip install -q \
    diffusers==0.29.1 \
    transformers>=4.41.0,<5.0.0 \
    accelerate==0.29.0 \
    huggingface_hub>=0.25.0 \
    safetensors==0.4.2 \
    gradio==4.29.0 \
    torch \
    bitsandbytes \
    xformers \
    websockets>=13.0,<15.0dev

## Import Dependencies

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline
from transformers import BitsAndBytesConfig
import gradio as gr
from PIL import Image

## Model Loading Helpers

In [ ]:
# 8-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
)
pipe = None      # base pipeline
refiner = None   # img2img refiner pipeline

def load_base_model():
    global pipe
    if pipe is None:
        print("Loading SDXL base model...")
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config,
        )
        try:
            pipe.enable_xformers_memory_efficient_attention()
            print("✅ xFormers attention enabled")
        except Exception as e:
            print("⚠️ xFormers unavailable:", e)

def load_refiner_model():
    global refiner
    if refiner is None:
        print("Loading SDXL refiner model...")
        refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-refiner-1.0",
            torch_dtype=torch.float16,
            device_map="auto",
            quantization_config=bnb_config,
        )
        try:
            refiner.enable_xformers_memory_efficient_attention()
            print("✅ xFormers attention for refiner enabled")
        except Exception as e:
            print("⚠️ xFormers unavailable for refiner:", e)


## Generation Function

In [ ]:
def generate_image(
    prompt: str,
    negative_prompt: str = "",
    num_steps: int = 30,
    guidance_scale: float = 7.5,
    use_refiner: bool = False,
    refiner_steps: int = 10,
    lora_model: str = "",
    lora_scale: float = 0.5,
    advanced_mode: bool = False,
) -> Image.Image:
    try:
        load_base_model()

        # LoRA loading
        cross_attention_kwargs = {}
        if advanced_mode and lora_model.strip():
            try:
                pipe.unet.load_attn_procs(lora_model)
                cross_attention_kwargs = {"scale": lora_scale}
                print(f"✅ LoRA loaded: {lora_model} @ {lora_scale}")
            except Exception as e:
                print(f"❌ LoRA error: {e}")
                return Image.new("RGB", (1024, 1024), color="black")

        # Base generation
        img = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=num_steps,
            guidance_scale=guidance_scale,
            cross_attention_kwargs=cross_attention_kwargs,
        ).images[0]

        # Refiner
        if advanced_mode and use_refiner:
            load_refiner_model()
            img = refiner(
                prompt=prompt,
                negative_prompt=negative_prompt,
                num_inference_steps=refiner_steps,
                image=img,
            ).images[0]

        return img

    except Exception as e:
        print(f"❌ Generation error: {e}")
        return Image.new("RGB", (1024, 1024), color="black")


## Gradio Interface

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown(
        "# 🎨 Stable Diffusion XL\nGenerate high-quality images from text with SDXL 1.0."
    )

    with gr.Row():
        with gr.Column(scale=2):
            prompt = gr.Textbox(
                label="Prompt",
                placeholder="A futuristic cityscape at dusk",
                lines=2,
            )
            negative_prompt = gr.Textbox(
                label="Negative Prompt",
                placeholder="blurry, low quality",
                lines=1,
            )

            num_steps = gr.Slider(1, 100, value=30, label="Inference Steps")
            guidance_scale = gr.Slider(1.0, 20.0, value=7.5, label="Guidance Scale")

            advanced = gr.Checkbox(label="Advanced Mode", value=False)
            use_refiner = gr.Checkbox(label="Use Refiner", value=False)
            refiner_steps = gr.Slider(1, 50, value=10, label="Refiner Steps")
            lora_model = gr.Textbox(label="LoRA Model ID", placeholder="HF repo path or local folder")
            lora_scale = gr.Slider(0.0, 2.0, value=0.5, label="LoRA Scale")

            advanced.change(
                fn=lambda t: gr.update(visible=t),
                inputs=advanced,
                outputs=[use_refiner, refiner_steps, lora_model, lora_scale],
            )

            btn = gr.Button("Generate Image")

        with gr.Column(scale=2):
            output = gr.Image(label="Generated Image", type="pil")

    btn.click(
        fn=generate_image,
        inputs=[
            prompt, negative_prompt, num_steps,
            guidance_scale, use_refiner, refiner_steps,
            lora_model, lora_scale, advanced,
        ],
        outputs=output,
    )

demo.launch(share=True)
